In [1]:
import torch
from torch_scatter import scatter

In [2]:
x = torch.randn((9, 4))
cluster = torch.cat((torch.zeros(4), torch.ones(5))).long()
print(x)
print(cluster)

tensor([[ 0.1266, -1.2514, -0.4005, -0.5617],
        [ 0.7433,  1.0250, -0.2480,  0.5520],
        [ 0.3471,  0.3888,  0.1874, -0.4121],
        [ 1.0442, -0.9976,  0.3196, -0.9103],
        [-0.6950,  1.4699, -1.3755, -0.9901],
        [ 0.0303, -1.4798,  1.1173,  0.5543],
        [-0.3560, -1.0236, -1.9061,  0.6711],
        [ 0.3916, -0.1797, -0.9383, -0.4893],
        [-1.9653,  0.2235, -0.1763,  2.2131]])
tensor([0, 0, 0, 0, 1, 1, 1, 1, 1])


In [5]:
c = scatter(x, cluster, dim=0, reduce='max')
print(c)
print(c.shape)
print(c[cluster])

tensor([[1.0442, 1.0250, 0.3196, 0.5520],
        [0.3916, 1.4699, 1.1173, 2.2131]])
torch.Size([2, 4])
tensor([[1.0442, 1.0250, 0.3196, 0.5520],
        [1.0442, 1.0250, 0.3196, 0.5520],
        [1.0442, 1.0250, 0.3196, 0.5520],
        [1.0442, 1.0250, 0.3196, 0.5520],
        [0.3916, 1.4699, 1.1173, 2.2131],
        [0.3916, 1.4699, 1.1173, 2.2131],
        [0.3916, 1.4699, 1.1173, 2.2131],
        [0.3916, 1.4699, 1.1173, 2.2131],
        [0.3916, 1.4699, 1.1173, 2.2131]])


In [4]:
def scatter_naive1D(src, index, dim, dim_size, reduce):
    # Assume 1D tensor. 
    assert dim == 0 and src.dim() == 1 and index.dim() == 1, "Only support 1D scatter!"

    def reduce_op(l, op):
        result = 0
        if op == 'min':
            result = torch.min(l)
        elif op == 'max':
            result = torch.max(l)
        elif op == 'sum':
            result = torch.sum(l)
        elif op == 'mean':
            result = torch.mean(l)
        else:
            assert False, 'Reduce op not supported!'

        return result 

    output = torch.zeros(dim_size, dtype=src.dtype, device=src.device)

    index_range = torch.max(index) + 1
    bucket = [[] for _ in range(index_range)]

    for i in range(len(src)):
        bucket[index[i]].append(src[i])

    for i in range(len(bucket)):
        if bucket[i]:
            bucket[i] = reduce_op(torch.as_tensor(bucket[i], dtype=src.dtype, device=src.device), reduce)
        else:
            bucket[i] = 0

    output[:len(bucket)] = torch.as_tensor(bucket, dtype=src.dtype, device=src.device)

    return output

In [9]:
scatter_naive1D(torch.tensor([19,2,3,4,5]), torch.tensor([0,0,1,1,1]), 0, (2,), 'max')

tensor([19,  5])

In [23]:
a = torch.tensor([[ 0.3694,  0.7417,  0.5024,  0.9618, -0.0140,  0.8021, -0.6510,  0.7326,
          0.7800,  1.5315,  0.3143,  2.6228,  0.5350,  0.7634, -0.4102,  1.1273,
         -0.0379,  1.1283,  0.8948,  0.2884,  0.1164,  0.2915,  0.7639,  0.7709,
         -0.1824,  1.7538, -0.4470,  1.0834,  0.9018,  0.3904,  0.1982,  1.3424,
          0.1479, -0.0301, -0.9842,  0.1113,  0.1695,  1.4892, -0.3998,  1.4559,
         -0.3716,  1.3200,  0.1965,  0.8607,  0.5246,  2.0542, -0.4456,  1.3815,
          0.2840,  1.6265,  0.8468,  0.8781,  0.1550,  0.6311,  0.4216,  0.4718,
          0.8718,  0.4823,  0.4982,  0.5845]]).view(1, 30, 2)

In [25]:
a.shape

torch.Size([1, 30, 2])

In [26]:
a

tensor([[[ 0.3694,  0.7417],
         [ 0.5024,  0.9618],
         [-0.0140,  0.8021],
         [-0.6510,  0.7326],
         [ 0.7800,  1.5315],
         [ 0.3143,  2.6228],
         [ 0.5350,  0.7634],
         [-0.4102,  1.1273],
         [-0.0379,  1.1283],
         [ 0.8948,  0.2884],
         [ 0.1164,  0.2915],
         [ 0.7639,  0.7709],
         [-0.1824,  1.7538],
         [-0.4470,  1.0834],
         [ 0.9018,  0.3904],
         [ 0.1982,  1.3424],
         [ 0.1479, -0.0301],
         [-0.9842,  0.1113],
         [ 0.1695,  1.4892],
         [-0.3998,  1.4559],
         [-0.3716,  1.3200],
         [ 0.1965,  0.8607],
         [ 0.5246,  2.0542],
         [-0.4456,  1.3815],
         [ 0.2840,  1.6265],
         [ 0.8468,  0.8781],
         [ 0.1550,  0.6311],
         [ 0.4216,  0.4718],
         [ 0.8718,  0.4823],
         [ 0.4982,  0.5845]]])

In [28]:
a.cumsum(1)

tensor([[[ 0.3694,  0.7417],
         [ 0.8718,  1.7035],
         [ 0.8578,  2.5056],
         [ 0.2068,  3.2382],
         [ 0.9868,  4.7697],
         [ 1.3011,  7.3925],
         [ 1.8361,  8.1559],
         [ 1.4259,  9.2832],
         [ 1.3880, 10.4115],
         [ 2.2828, 10.6999],
         [ 2.3992, 10.9914],
         [ 3.1631, 11.7623],
         [ 2.9807, 13.5161],
         [ 2.5337, 14.5995],
         [ 3.4355, 14.9899],
         [ 3.6337, 16.3323],
         [ 3.7816, 16.3022],
         [ 2.7974, 16.4135],
         [ 2.9669, 17.9027],
         [ 2.5671, 19.3586],
         [ 2.1955, 20.6786],
         [ 2.3920, 21.5393],
         [ 2.9166, 23.5935],
         [ 2.4710, 24.9750],
         [ 2.7550, 26.6015],
         [ 3.6018, 27.4796],
         [ 3.7568, 28.1107],
         [ 4.1784, 28.5825],
         [ 5.0502, 29.0648],
         [ 5.5484, 29.6493]]])